# 1. 分析过程
#### 识别客户价值应用最广泛的模型是通过3个指标（最近消费时间间隔（Recency）、消费频率（Frequency）和消费金额（Money））来进行客户细分，识别出高价值客户，简称RFM模型。

RFM模型中，消费金额表示一段时间内，客户购买企业产品金额的总和。由于航空票价受到运输距离、舱位等级等多种因素影响，同样消费金额的不同旅客对航空公司的价值是不同的。例如，一位购买长航线、低等舱位票的旅客与一位购买短航线、高等级舱位票的旅客相比，后者对于航空公司的价值可能更高。因此，这个指标并不适合航空公司客户价值分析。我们选择客户在一定时间内累积的飞行里程M和客户在一定时间内乘坐舱位所对应的折扣系数的平均值C两个指标代替消费金额。此外，还考虑航空公司会员入会时间的长短在一定程度上影响客户价值，所以在模型中增加客户关系长度L，作为区分客户的另一指标。


综上所述，航空公司识别客户价值模型（LRFMC）指标如下：


#### 客户关系长度L：会员入会时间距观测窗口结束的月份

#### 消费时间间隔R：客户最近一次乘坐公司飞机距观测窗口结束的月数

#### 消费频率F：客户在观测窗口内乘坐公司飞机的次数

#### 飞行里程M：客户在观测窗口内飞行里程

#### 折扣系数的平均值C：客户在观测窗口内乘坐舱位所对应的折扣系数的平均值


 传统的RFM模型采用属性分箱的方法，如下图所示（依据属性的平均值进行划分），虽然也能识别出最有价值客户，但是如果LRFMC模型使用同样的方法，则细分的属性太多，提高了后续根据客户类别的营销成本。故本次分析使用聚类方法，通过对航空公司客户价值的五个指标进行Kmeans聚类分析，识别出最有价值客户。


### 航空公司客户价值信息挖掘主要包話以下步骤：


（1）从航空公司的数据源中进行选择性抽取与新增数据抽取分别形成历史数据和增量数据；

（2）对步骤1）中形成的两个数据集进行数据探索分析和预处理，包括数据缺失值和异常值分析，数据属性的规约、清洗和变换；

（3）利用步骤2）中的处理的数据进行建模，基于旅客价值的LRFMC模型进行客户分类，对各个客户群进行特征分析，识别出最有价值的客户；

（4）针对模型结果得到不同价值的客户，采用不同的营销手段，提供定制化的服务。

# 2. 数据探索分析

针对本项目的数据，主要进行缺失值分析和异常值分析。比如票价为空值，票价最小值为0、折扣率最小值为、总飞行里程数大于零的记录。
拿到原始数据后，可以使用Python代码实现对数据的初步分析，主要使用pandas库中的describe()函数，代码如下：

In [13]:
# 对数据进行基本的探索，返回缺失值个数以及最大值最小值等
import pandas as pd
 
datafile = "./data/air_data.csv"  #原始数据
resultfile = "./tmp/explore.xls"   #数据探索结果总结表
 
data = pd.read_csv(datafile,encoding = "utf-8")  #读取原始数据，指定UTF-8编码（需要用文本编辑器将数据装换为UTF-8编码）

# 包括对数据的基本描述，percentiles参数是指定计算多少的分位数表（如1/4分位数、中位数等）；T是转置，转置后更方便查
explore = data.describe(percentiles=[],include="all").T
 
#describe()函数自动计算非空值数，需要手动计算空值数
explore["null"] = len(data) - explore["count"]
 
explore = explore[["null","max","min"]]
explore.columns = [u"空数值",u"最大值",u"最小值"] #对结果统计的表头重命名

# 这里只选取部分探索结果。
# describe()函数自动计算的字段有count（非空值数）、unique（唯一值数）、top（频数最高者）、
# freq（最高频数）、mean（平均值）、std（方差）、min（最小值）、50%（中位数）、max（最大值）
# print(explore)
 
explore.to_excel(resultfile) ##导出结果

# 3. 数据预处理

### 针对上述问题的数据预处理包括数据清洗、属性规约和数据变换。



## 3.1数据清洗
由上述分析可知，原始数据中存在着票价为空值，票价最小值为0、折扣率最小值为0而总飞行里程数大于零的等异常记录。考虑原始数据样本量较大，而这类数据的数量又很小，因此可以选择舍弃处理。

1.舍弃票价为空的记录；

2.舍弃票价为零、平均折扣率不为零，总飞行公里大于零的数据。

同样适用Python的pandas库进行数据清洗，代码如下所示：

In [16]:
#票价非空值保留
data=data[data['SUM_YR_1'].notnull()*data['SUM_YR_2'].notnull()]

#只保留非空票价，或者平均值扣率与总飞行公里数同时为0的记录
index1=data['SUM_YR_1']!=0
index2=data['SUM_YR_2']!=0
index3=(data['SEG_KM_SUM']==0) & (data['avg_discount']==0)

data=data[index1|index2|index3]

/Library/Python/3.8/site-packages/pandas/core/computation/expressions.py:200: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


## 3.2 属性归约

原始数据中有众多属性，需要选取与本次模型相关的属性，最终选取的6个相关属性分别为：入会时间（FFP_DATE）、观测窗口的结束时间（LOAD_TIME）、飞行次数（FLIGHT_COUNT）、平均折扣率（AVG_DISCOUNT）、观测窗口总飞行里程数（SEG_KM_SUM）、最后一次乘机时间至观察窗口末端时长（LAST_TO_END）。删除与模型不相关、弱相关或冗余的属性，比如，会员卡号、性别、工作地城市、工作地所在省份等

## 3.3 数据变换

数据变换，即将数据变换为“适当的”格式，以适应挖掘任务以及算法的需求。本项目主要的数据变换方式为属性构造和数据标准化。

### 3.3.1 属性构造

由于数据中并没有直接给出LRFMC5个指标，需要通过属性构造来提取这5个指标。具体的计算方式如下：

（1）L=LOAD_TIME-FFP_DATE

（2）R=LAST_TO_END

（3）F=FLIGHT_COUNT

（4）M=SEG_KM_SUM

（5）C= AVG_DISCOUNT

### 3.3.2 数据标准化

得到LRFMC5个指标后，对这五个指标进行分析，发现五个指标取值范围数据差异较大，为了消除数量级数据带来的影响，需要对数据进行标准化处理，本次使用标准差标准化处理，代码如下所示：

In [19]:
# -*- coding:utf-8 -*-
#标准差标准化
  
datafile = "./data/zscoredata.xls"
zscorefile = "./tmp/zscoreddata.xls"
 
#标准化处理
data = pd.read_excel(datafile)
data = (data - data.mean(axis=0))/(data.std(axis=0))
 
data.columns = ["Z"+i for i in data.columns] ##表头重新命名，前面加上Z
 
data.to_excel(zscorefile,index=False)

# 4. 模型构建

采用KMeans聚类算法对客户数据进行客户分群，结合业务相关知识确定聚成5类客户。

Kmeans聚类算法位于Sklearn库下的聚类子库（sklearn.cluster），代码如下所示：

In [26]:
# -*- coding:utf-8 -*-
 
#KMeans聚类方法
 
import pandas as pd    
from sklearn.cluster import KMeans
 
 
inputfile = "./data/zscoreddata.xls"
 
 
k = 5 #聚类为5类
 
 
data = pd.read_excel(inputfile)
 
    
#调用KMeans方法，进行分析
#n_job为并行数，设定为CPU数目较好
kmodel = KMeans(n_clusters=k,n_jobs=1) 
 
kmodel.fit(data) #训练模型
 
 
 
#查看聚类中心以及聚类数目
 
r1=pd.Series(kmodel.labels_).value_counts()
 
r2=pd.DataFrame(kmodel.cluster_centers_)
 
r=pd.concat([r2,r1],axis=1)
 
r.columns=list(data.columns)+['类别数目']
 
 
 
print(r)

/Library/Python/3.8/site-packages/sklearn/cluster/_kmeans.py:938: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn("'n_jobs' was deprecated in version 0.23 and will be"


         ZL        ZR        ZF        ZM        ZC   类别数目
0  0.483578 -0.799801  2.482604  2.423403  0.310271   5342
1  1.161276 -0.377376 -0.086756 -0.094717 -0.159957  15717
2 -0.701007 -0.415247 -0.160076 -0.159691 -0.259283  24587
3 -0.313417  1.687409 -0.573858 -0.536703 -0.174168  12103
4  0.044216 -0.003467 -0.235869 -0.239966  2.155835   4295


# 5. 客户价值分析

<img src="./客户雷达图.png" width="60%">
<img src="./客户柱状图.png" width="90%">

针对聚类结果进行特征分析，其中

              优势	 劣势	      数目
    客户群1	C	    R、F、M	  最少 5
    客户群2	 	    C、L	     最多1
    客户群3	 	    R、M、F	  3
    客户群4	L、F、M	 	      2
    客户群5	M、F、R	 	      4



### 分析：

客户群1的数量最少，所乘航班折扣率较高（一般所乘航班的等级较高），属于重要发展客户。

客户群2的数量较多，所乘航班折扣率较低，加入会员时间短，这类客户一般在打折时才会乘坐航班，属于一般客户。

客户群3，乘坐次数很少，累计总飞行公里数较少，很久没有乘坐飞机，属于低价值客户。

客户群4的数量较多，加入会员时间长，但是最近乘坐频率变小，属于重要挽留客户。

客户群5，累计总飞行公里数较大，飞行次数较多，最近乘坐过飞机，属于重要保持客户。
 
 



### 客户价值排名：

    客户群	排名	排名含义

    客户群5	1	重要保持客户

    客户群1	2	重要发展客户
    
    客户群4	3	重要挽留客户

    客户群2	4	一般客户

    客户群3	5	低价值客户


其中重要发展客户、重要保持客户、重要挽留客户分别对应客户生命周期管理的发展期、稳定期、衰退期。
 
## 结论：
### 根据对每个客户群进行特征分析，采取针对性营销：
### 1. 会员升级与保级--客户群4（数量较大）
### 2. 首次兑换--客户群2
### 3.交叉销售--客户群5